# TODOs
- [X] Metadaten.json zu Python Variablen, in Python Dictionary einspeisen und auslesen (Parkplätze, WiFi) -> Frank
- [X] Map (Slider, Heat Map, Animationen) Visualisierungen -> Frank
- [X] virtual env vorbereiten -> Rich
- [X] projektserver? -> Rich
- [X] Erste Statistiken (Bar Charts, Graphs usw.) Visualisierungen -> Rich
- [ ] Chart Ideen -> Anja
- [ ] Unsere Daten in PowerBI einspeisen, (welche Daten bräuchten wir z.B. für Graph Bubble oder allgemeine Sachen) oder was kostenloses (z.B. CliqSense) -> Anja

# Unsere Quellen zum Visualisierungserfolg:
### Wie Wähle ich die richtige Visualisierung?

https://eazybi.com/blog/data_visualization_and_chart_types/

### Wo finde ich sie in Python?

https://python-graph-gallery.com

# Domänenwissen

* Öffnungszeiten (Theater, Uni)
* Peak Times, Tag
* Ferienzeiten
* Special Events (Demos, Strassensperren)
* Parkplätze kostenlos/bezahlt?
* Sonntag Flohmarkt

# Ideen
* Vergleich mit Wetterdaten
* Verlgeich mit Öffis -> Empfehlungen aussprechen -> z.B. mehr Busse sollen fahren, aber eine U-Bahn kann ausfallen
* Anhand von hauptsächlich Fußgänger könnte man behaupten, wann Vorlesungen auf der TU sind

# Korrelation
* Wie viele fahren rein um bestimmte Uhrzeit rein -> Belegung Parkplätze
* WiFi Nutzung in Zusammenhang mit parkenden Autos?

# Prädiktionen (ML)
* Traffic-/Personenaufkommen zu bestimmten Tageszeiten

# Distribution
* Normalverteilung

# Intervall
* Tageszeiten
* Wochen

# Fragen flow.lab
* Wie wurden Pedestrian Daten aufgenommen?
* Was sind die Daten bei WiFi?
* Gibt es schon Ergebnisse aus Wettbewerb?
* Gibt es neue Daten?
* Erklärung zu Datenset?

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import folium.plugins as plugins
from IPython.display import HTML
import numpy as np
import json

### Erklärungen/Links zu wichtigen Libraries
- Markdown schreiben hier in Jupyter: https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet
- folium für map Visualisierungen: https://github.com/python-visualization/folium

### Daten einlesen
> Erns-Reuter-Platz Datensatz aus http://flow.dai-labor.de

In [4]:
wifi = pd.read_csv('data/wifi.csv')
parking = pd.read_csv('data/parking.csv', delimiter=' ')
# TODO: hier restlichen Datensätze einlesen

### Sinn aus den Daten machen 

In [5]:
# TODO: alle Typen aus der metadata.json in Python Variablen, z.B.:
campus_eb = [52.512388, 13.32360925]
campus_tel = [52.51298775, 13.32029525]

# TODO: json daten auslesen
metadata = json.loads(open('data/metadata.json').read())

wifi_latlon = metadata['wifi']['latlon']
pedestrian_latlon = metadata['pedestrian']['latlon']
traffic_latlon = metadata['traffic']['latlon']
parking_latlon = metadata['parking']['latlon']

### Wichtige Libraries
- folium für map Visualisierungen: https://github.com/python-visualization/folium

### Alle WiFi Standorte auf Karte visualisieren

In [6]:
## TODO: take center of ernst-reuter platz as location=POINT
map = folium.Map(location=campus_eb, zoom_start=16)


for key in wifi_latlon:
    folium.Circle(
        radius=50,
        location=wifi_latlon[key],
        popup=key,
        color='#3186cc',
        fill=True
    ).add_to(map)
    
for key in pedestrian_latlon:
    folium.Marker(
        location=pedestrian_latlon[key],
        popup=key,
        icon=folium.Icon(icon='info-sign')
    ).add_to(map)
    
for key in traffic_latlon:
    folium.Marker(
        location=traffic_latlon[key],
        popup=key,
        icon=folium.Icon(color='red',icon='info-sign')
    ).add_to(map)

for key in parking_latlon:
    folium.Circle(
        radius=2,
        location=parking_latlon[key],
        popup=key,
        color='red',
        fill=True
    ).add_to(map)

map

### Zum Vergleich Originalstandorte:
(TODO: Karte rotieren?)

![Sensor_Locations](images/Sensor_Locations.jpg)

### Parking Data

In [7]:
mean_parking = {}

startTime = float(parking[['Timestamp']].iloc[0])
endTime = float(parking[['Timestamp']].iloc[-1])
timeSpan = endTime - startTime

for key in parking_latlon:
    temp = parking[parking['ParkingSpotID'] == key][['Timestamp','Occupied']]
    occupied = temp.iloc[0]['Occupied'] == 1
    lastTime = startTime
    mean_parking[key] = 0
    for index, row in temp.iterrows():
        if occupied != (row['Occupied'] == 1):
            if occupied:
                mean_parking[key] += row['Timestamp'] - lastTime
            else:
                lastTime = row['Timestamp']
            occupied = row['Occupied'] == 1
    if occupied:
                mean_parking[key] += endTime - lastTime
    mean_parking[key] /= timeSpan
    print(key, ":", mean_parking[key])
                
        
    
## mean_parking[key] = parking[parking['ParkingSpotID'] == key][['Occupied']].mean()    
## This is the wrong aproach! data points are only created when the Occupied property changes, so the Mean has to be around 50%
## the correct way is to calculate the mean-time the Parking Spot was occupied

##print(mean_parking)


LZQ2AMNPAAAAU7QDBHVMEVVZ : 0.4745755398860814
LZQ2AMXWAAAAU7QDBHFOWJM5 : 0.5073588844292795
LZQ2AN4NAAAAU7QDBHQGNAFN : 0.44496773662609473
LZQ2ANAVAAAAU7QDBFBNCMYI : 0.40650217531453403
LZQ2ANJGAAAAU7QDBFCQE74V : 0.3927269087100178
LZQ2ANSFAAAAU7QDBHGHYITI : 0.5066861406338022
LZQ2AO5NAAAAU7QDBHHNGG7J : 0.3959405832415216
LZQ2AOEPAAAAU7QDBHZM5ZYB : 0.30792665329694974
LZQ2AOMUAAAAU7QDBHBEY6HX : 0.4171694467749902
LZQ2AOVHAAAAU7QDBF2FWKNL : 0.35652623531929434
LZQ2AP7HAAAAU7QDBENVJF67 : 0.2668880001103557
LZQ2APGFAAAAU7QDBHUW4O3U : 0.3705665578094636
LZQ2APPBAAAAU7QDBG66NGI4 : 0.41937895407675896
LZQ2APWZAAAAU7QDBE74KH4Y : 0.37209460567403124
LZQ2AQHIAAAAU7QDBFQAPZRO : 0.5293633821403173
LZQ2AQPOAAAAU7QDBEOQ7GRC : 0.43051202648294506
LZQ2AQYZAAAAU7QDBHZLKV4Z : 0.3735082130365179
LZQ2ARBLAAAAU7QDBHHHZXBS : 0.43668823875065543
LZQ2ARKXAAAAU7QDBH3D725J : 0.41271667139640544
LZQ2ARSCAAAAU7QDBFQXMEGM : 0.4293735834663292
LZQ2ARZTAAAAU7QDBEBS7S4V : 0.42231784320333643
LZQ2ASBXAAAAU7QDBFRTOMVS

In [8]:
parking_map = folium.Map(location=campus_eb, zoom_start=16)

for key in parking_latlon:
    visual = (mean_parking[key] - 0.25) * 2
    folium.Circle(
        radius=1,
        location=parking_latlon[key],
        popup=key,
        color='#%02x%02x%02x' % (int(255 * visual), 0, int(255 * (1 - visual))),
        fill=True
    ).add_to(parking_map)
    
## rot = häufig belegt, blau = weniger häufig belegt

parking_map